In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import os

In [2]:
SUBMIT_FOLDER = "../stage1_submit"

In [3]:
df_best = pd.read_csv(SUBMIT_FOLDER + '/' + 'submit_ensemble_3e79a0_revised.csv')
df_best.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,25


In [4]:
df_null = df_best.loc[df_best['Predicted'].isnull()]
df_null.head()

,Id,Predicted
14,00407c16-bad3-11e8-b2b8-ac1f6b6435d0,NaN
44,00de1ab2-bad2-11e8-b2b8-ac1f6b6435d0,NaN
45,00deeaca-bada-11e8-b2b9-ac1f6b6435d0,NaN
47,00e00578-baca-11e8-b2b8-ac1f6b6435d0,NaN
48,00e00666-badb-11e8-b2b9-ac1f6b6435d0,NaN


In [5]:
#pred_list = [f for f in os.listdir(SUBMIT_FOLDER)]
pred_list = [
    'submit_model_kf4_6f1a27.csv',
    'submit_model_a966cd.csv',
    'submit_model_kf4_b1a73d.csv',
    'submit_model_kf1_417591.csv',
    'submit_model_kf4_b58461.csv',
    'submit_model_kf2_155691.csv'
]

#pred_list = ['submit_model_a966cd.csv', 'submit_model_kf4_6f1a27.csv','submit_model_kf4_b1a73d.csv']
for i in range(len(pred_list)):
    print ('{} -> {}'.format(i, pred_list[i]))

0 -> submit_model_kf4_6f1a27.csv
1 -> submit_model_a966cd.csv
2 -> submit_model_kf4_b1a73d.csv
3 -> submit_model_kf1_417591.csv
4 -> submit_model_kf4_b58461.csv
5 -> submit_model_kf2_155691.csv


In [6]:
df_join = df_null
for p in pred_list[:10]:
    df = pd.read_csv(SUBMIT_FOLDER + '/' + p)
    df_join = pd.merge(df_join, df, how='inner', on='Id')
df_join.head()

,Id,Predicted_x,Predicted_y,Predicted_x,Predicted_y,Predicted_x,Predicted_y,Predicted
0,00407c16-bad3-11e8-b2b8-ac1f6b6435d0,NaN,NaN,25,NaN,0 25,NaN,15 25
1,00de1ab2-bad2-11e8-b2b8-ac1f6b6435d0,NaN,NaN,NaN,NaN,25,NaN,15 17 25
2,00deeaca-bada-11e8-b2b9-ac1f6b6435d0,NaN,NaN,NaN,23,NaN,NaN,15
3,00e00578-baca-11e8-b2b8-ac1f6b6435d0,NaN,NaN,0,NaN,0,25,15 25
4,00e00666-badb-11e8-b2b9-ac1f6b6435d0,NaN,6,6 11,6,6,11,6 15


In [7]:
df_join = pd.merge(df_null, df, how='inner', on='Id')
df_join.head()

,Id,Predicted_x,Predicted_y
0,00407c16-bad3-11e8-b2b8-ac1f6b6435d0,NaN,15 25
1,00de1ab2-bad2-11e8-b2b8-ac1f6b6435d0,NaN,15 17 25
2,00deeaca-bada-11e8-b2b9-ac1f6b6435d0,NaN,15
3,00e00578-baca-11e8-b2b8-ac1f6b6435d0,NaN,15 25
4,00e00666-badb-11e8-b2b9-ac1f6b6435d0,NaN,6 15


In [8]:
df_join['Predicted_x'].fillna(df_join['Predicted_y'], inplace=True)
df_join.head()

,Id,Predicted_x,Predicted_y
0,00407c16-bad3-11e8-b2b8-ac1f6b6435d0,15 25,15 25
1,00de1ab2-bad2-11e8-b2b8-ac1f6b6435d0,15 17 25,15 17 25
2,00deeaca-bada-11e8-b2b9-ac1f6b6435d0,15,15
3,00e00578-baca-11e8-b2b8-ac1f6b6435d0,15 25,15 25
4,00e00666-badb-11e8-b2b9-ac1f6b6435d0,6 15,6 15


In [9]:
def make_label_array (labels):

    # set up a numpy array to receive the encoded label
    label_array = np.zeros(28)

    if isinstance(labels, str):
    
        # split the space separated multi-label into a list of individual labels
        split_label = labels.split(' ')

        # turn on the positive columns in the labels array
        for label in split_label:
            label_array[np.uint8(label)] = 1
    
    return label_array
    
# unit test 
la = make_label_array(df_best['Predicted'][0])
print(la.shape)
print(la)


(28,)
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [10]:
pred_list = [
    'submit_model_kf4_6f1a27.csv',
    'submit_model_a966cd.csv',
    'submit_model_kf4_b1a73d.csv',
    'submit_model_kf1_417591.csv',
    'submit_model_kf4_b58461.csv',
    'submit_model_kf2_155691.csv',
    'submit_model_kf3_60f138.csv'
]

for p in pred_list:
    this_prediction = np.zeros(28)
    df = pd.read_csv(SUBMIT_FOLDER + '/' + p)   
    print(df['Predicted'][0])
    this_prediction = make_label_array(df['Predicted'][0])
    print(this_prediction)
#    sum_of_predictions += this_prediction


2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2 15
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2 27
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1.]


In [11]:
pred_list = [
    'submit_model_kf4_6f1a27.csv',
    'submit_model_a966cd.csv',
    'submit_model_kf4_b1a73d.csv',
    'submit_model_kf1_417591.csv',
    'submit_model_kf4_b58461.csv'
]

# loop through each prediction
for i in tqdm(range(len(df_best))):
    
    # if the prediction is null
    if not type(df_best['Predicted'][i]) is str:
        
        # set up an array to sum up the value of predictions
        sum_of_predictions = np.zeros(28)
        
        # loop through each prediction set, convert to a numpy array, sum up the votes
        for p in pred_list:
            this_prediction = np.zeros(28)
            df = pd.read_csv(SUBMIT_FOLDER + '/' + p)   
            this_prediction = make_label_array(df['Predicted'][i])
            sum_of_predictions += this_prediction
        
        # set up the prediction string
        predict_str = ''
        for j in range(sum_of_predictions.shape[0]):
                if sum_of_predictions[j] > 1:
                    predict_str += str(j) + ' '
        if type(predict_str) is str: 
            df_best['Predicted'][i] = predict_str.strip()

100%|██████████| 11702/11702 [00:25<00:00, 459.64it/s]


In [12]:
df_null = df_best.loc[df_best['Predicted'].isnull()]
print(len(df_null))
#df_null.head()

0


In [13]:
df_best[df_best['Id'] == 'fcfaae16-bacb-11e8-b2b8-ac1f6b6435d0']

,Id,Predicted
11558,fcfaae16-bacb-11e8-b2b8-ac1f6b6435d0,0 21 25


In [14]:
df_best.to_csv(SUBMIT_FOLDER + '/' + 'submit_ensemble_3e79a0_voting.csv', index=False)